<a href="https://colab.research.google.com/github/Charan-Gond/Blue-Bus/blob/main/08_nlp_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Introduction to NLP
  another common term for NLP is sequence to sequence (seq2seq).

In [1]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


## get helper function

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2025-09-05 16:46:19--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0.001s  

2025-09-05 16:46:19 (18.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## get text dataset

the dataset using is kagel intriduction to NLP datasets (text sample of tweets (disaster or not disaster))

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# unzip data
unzip_data("nlp_getting_started.zip")

--2025-09-05 16:46:29--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.181.207, 142.251.184.207, 64.233.179.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.181.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.008s  

2025-09-05 16:46:29 (73.8 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## visulizing text data
one way using python
https://realpython.com/read-write-files-python/

other way pandas

In [4]:
import pandas as pd
train_df=pd.read_csv("train.csv");
test_df=pd.read_csv("test.csv");
train_df.head()


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df["text"][1]

'Forest fire near La Ronge Sask. Canada'

In [6]:
# shuffle training data

train_df_shuffled=train_df.sample(frac=1,random_state=42)

train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# how many example are there

train_df_shuffled.target.value_counts()

,count
target,
0,4342
1,3271


In [9]:
# how many total samples
len(train_df_shuffled),len(test_df)

(7613, 3263)

In [10]:
# lets visulize random training example
import random
random_index =random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target=row
  print(f"Target: {target}"," (real distarer)" if target>0 else "(not a distaer)")
  print(f"Test\n{text}")
  print("---\n")

Target: 0 (not a distaer)
Test
@MeekMill is w(rec)k league ball @Drake is Olympic level body bagging him like his career was nothing #trollingtilMeekdiss
---

Target: 1  (real distarer)
Test
We're #hiring! Click to apply: RN II/EMERGENCY SERVICES/FT/7P-7A - http://t.co/NV3Uxv9IMX #Nursing #Houston TX http://t.co/ej30IhrEA9
---

Target: 0 (not a distaer)
Test
Anti Collision Rear- #innovation #gadgets http://t.co/SXQTydUvUL
---

Target: 0 (not a distaer)
Test
New #photo Oak in a snowstorm http://t.co/JhSCGDA2G8 on the #SouthDowns #Hampshire #Winter #photography #art #tree #treescape #treeporn
---

Target: 0 (not a distaer)
Test
Û÷RansomwareÛª holds B.C. manÛªs computer files hostage: A virus locked Andrew Wilson's family photos and otherÛ_ http://t.co/aQbLjComlN
---



### split data into training and validate

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train_sentences, val_sentences, train_label, val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                         train_df_shuffled["target"].to_numpy(),
                                                                           test_size=0.1,random_state=42)

In [13]:
len(train_sentences), len(train_label),len(val_sentences),len(val_labels)

(6851, 6851, 762, 762)

In [14]:
train_sentences[:10], train_sentences[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array(['@

## convert text to number

### Text vectorization (tokenization)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# use deafult textVectorrization

text_vectorizer=TextVectorization(max_tokens=None,
                                  standardize='lower_and_strip_punctuation',
                                  split="whitespace",
                                  ngrams=None,
                                  output_mode='int',
                                  output_sequence_length=None,
                                  # pad_to_max_tokens=True
                                  )

In [16]:
# find the average number of tokens (words) in the training tweets

round(sum([len(i.split()) for i in train_sentences ])/len(train_sentences))

15

In [17]:
max_vocab_length=10000 # max length of words to have in our vocabulary
max_length=15 # average

text_vectorizer=TextVectorization(max_tokens=max_vocab_length,
                                  output_mode='int',
                                  output_sequence_length=max_length,
                                  )

In [18]:
# Fit the text vectorizer to Train data
text_vectorizer.adapt(train_sentences)

In [19]:
# create a sample sentence and tokenize it
sample_sentence="There is a flodd in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3,   1,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [20]:
#choose a random sample sentence fromthe training dataset and tokenize it

random_sentence=random.choice(train_sentences)


In [21]:
# get the uniqe words iin the vocabualry
words_in_vocab=text_vectorizer.get_vocabulary()
words_in_vocab[:5] # most common word

['', '[UNK]', np.str_('the'), np.str_('a'), np.str_('in')]

In [22]:
words_in_vocab[-5:] # least common word

[np.str_('pages'),
 np.str_('paeds'),
 np.str_('pads'),
 np.str_('padres'),
 np.str_('paddytomlinson1')]

### creating embeddings using Embedding layer

the parameter we cared about is
1 input_dim
2 output_dim
3 input_length

In [23]:
 from tensorflow.keras import layers

 embedding=layers.Embedding(input_dim=max_vocab_length,
                              output_dim=128,
                              input_length=max_length
                              )

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [24]:
embedding

<Embedding name=embedding, built=False>

In [25]:
# Get a random sentence from the tarining test
random_sentence=random.choice(train_sentences)
print(random_sentence)

# Embed the sentensce( convet into dense vector)
print("\nEmbedded version")
sample_embed=embedding(text_vectorizer([random_sentence]))

sample_embed


They've come back!! &gt;&gt; Flying ant day: Capital deluged by annual swarm of winged insects http://t.co/mNkoYZ76Cp

Embedded version


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00781778, -0.04449635,  0.04694221, ...,  0.03092598,
          0.00094354, -0.0158625 ],
        [-0.01969842, -0.04561936,  0.02165102, ..., -0.03117992,
         -0.00853501,  0.04068006],
        [ 0.03858228, -0.03069149, -0.01151484, ...,  0.03179891,
          0.00225805, -0.01206614],
        ...,
        [ 0.0334954 ,  0.02919718,  0.03835798, ...,  0.0333265 ,
          0.0240407 ,  0.0188182 ],
        [ 0.02871612, -0.03460785, -0.00778309, ...,  0.01241159,
          0.02001596, -0.0043753 ],
        [ 0.00742717, -0.04104728, -0.01124404, ..., -0.04879905,
          0.0164469 , -0.0433786 ]]], dtype=float32)>

In [26]:
# check out a single tokens embeddings

sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.00781778, -0.04449635,  0.04694221, -0.04973504, -0.04249312,
        -0.02928594, -0.01945626,  0.0254481 ,  0.00170439,  0.02438232,
        -0.01814556, -0.0485434 , -0.02580609,  0.02680154, -0.03556564,
        -0.00948857,  0.04062584,  0.04239731, -0.0289247 , -0.0076487 ,
        -0.04373334,  0.02942164,  0.02593038, -0.03710169, -0.03752923,
         0.03556608,  0.01590952, -0.03151973,  0.03043041,  0.04339786,
        -0.04326401,  0.01182754, -0.03775775, -0.045197  , -0.02378811,
         0.03193046, -0.04004068,  0.0435637 ,  0.03000924,  0.02836764,
         0.04573459,  0.029764  , -0.01923763,  0.02093203,  0.03487227,
         0.0200982 ,  0.01763189,  0.02449444, -0.02436629,  0.04131687,
        -0.04587512, -0.00444313,  0.02117688, -0.03253819, -0.0465479 ,
         0.04817576,  0.02072536, -0.01618141, -0.0032864 ,  0.01965478,
        -0.01097065,  0.02375467,  0.02383209, -0.04373721,  0.04784383,
  

## modeling a text dataset

we'll start with a baseline and move on from there

Model 0: Naive Bayes (baseline)
Model 1: Feed-forward neural network (dense model)
Model 2: LSTM model (RNN)
Model 3: GRU model (RNN)
Model 4: Bidirectional-LSTM model (RNN)
Model 5: 1D Convolutional Neural Network (CNN)
Model 6: TensorFlow Hub Pretrained Feature Extractor (useing trasfer learning)
Model 7: Same as model 6 with 10% of training data

### model 0 : getting base line
 we use sklearns's  multinomial aive nays using tF-IDF formula to convert words to number



In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# creating tokenization and modelling pipelne
model_0=Pipeline([
                  ("tfidf",TfidfVectorizer()),
                  ("clf",MultinomialNB())
])

# Filt the pipeline to the training data
model_0.fit(train_sentences,train_label)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [28]:
# Evaluate a model
baseline_score=model_0.score(val_sentences,val_labels)
{baseline_score*100}


{79.26509186351706}

In [29]:
# make predicttion
baseline_preds=model_0.predict(val_sentences)
baseline_preds[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0])

### creating an evaluating fuction for model experiments


In [30]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
def calculate_results(y_true,y_pred):
  """
  for binary classification
  """

  model_accuarcy=accuracy_score(y_true,y_pred)*100
  model_precision, model_recall, model_f1,_=precision_recall_fscore_support(y_true,y_pred,average="weighted")

  model_results={"acuuracy":model_accuarcy,
                 "precision":model_precision,
                 "recall": model_recall,
                 "f1":model_f1
                }

  return model_results

In [31]:
baseline_results=calculate_results(val_labels,baseline_preds)
baseline_results

{'acuuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: dense model

In [32]:
# create a tensorboard callback ()
from helper_functions import create_tensorboard_callback

SAVE_DIR="model_logs"

In [33]:
import numpy as np

In [34]:
# Build model with functional api
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimentianal strings

x=text_vectorizer(inputs) #convert string into number

x=embedding(x) # create an embedding nuberised inputs

x = layers.GlobalAveragePooling1D()(x) #condence the feature vector for ecah token to one vector

outputs=layers.Dense(1, activation="sigmoid")(x) # create output layer binary output use sigmoid

model_1=tf.keras.Model(inputs,outputs, name="model_1_dense")


In [35]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
#
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"],
                )

In [37]:
#fit
model_1_history = model_1.fit(x=train_sentences,
                              y=train_label,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")]
                              )

Saving TensorBoard log files to: model_logs/model_1_dense/20250905-164632
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 25ms/step - accuracy: 0.6437 - loss: 0.6501 - val_accuracy: 0.7454 - val_loss: 0.5410
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.8106 - loss: 0.4646 - val_accuracy: 0.7822 - val_loss: 0.4774
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.8582 - loss: 0.3586 - val_accuracy: 0.7887 - val_loss: 0.4624
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8893 - loss: 0.2925 - val_accuracy: 0.7848 - val_loss: 0.4645
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.9097 - loss: 0.2389 - val_accuracy: 0.7874 - val_loss: 0.4762


In [38]:
# check the result
model_1.evaluate(val_sentences,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7805 - loss: 0.5087


[0.47622039914131165, 0.787401556968689]

In [39]:
#make somee prediction and evaluate those
model_1_pred_probs=model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


(762, 1)

In [40]:
model_1_pred_probs[:5]

array([[0.3453181 ],
       [0.8368778 ],
       [0.9979129 ],
       [0.10903067],
       [0.12893708]], dtype=float32)

In [41]:
#predicting single text
model_1.predict(np.array([val_sentences[2]],dtype=object))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


array([[0.9979129]], dtype=float32)

In [42]:
model_1_preds=tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [43]:
model_1_results=calculate_results(val_labels,model_1_preds)

model_1_results

{'acuuracy': 78.74015748031496,
 'precision': 0.7923216200196284,
 'recall': 0.7874015748031497,
 'f1': 0.7844114973523961}

In [44]:
baseline_results

{'acuuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [45]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

## visulaize learned embedddings

In [46]:
# get the vocabulary from the text vectorrization layer
words_in_vocab =text_vectorizer.get_vocabulary()

len(words_in_vocab), words_in_vocab[:10]

(10000,
 ['',
  '[UNK]',
  np.str_('the'),
  np.str_('a'),
  np.str_('in'),
  np.str_('to'),
  np.str_('of'),
  np.str_('and'),
  np.str_('i'),
  np.str_('is')])

In [47]:
# moedel 1 summary
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,840,389 (14.65 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,560,260 (9.77 MB)

In [48]:
# get the weight metrix of the embedding layer
# numeric representation of each token (trained  5 epochs )
embed_weights=model_1.get_layer("embedding").get_weights()[0]

print(embed_weights.shape) # same asize vocab size and embedding dim(o/p dim)

(10000, 128)


In [49]:
embed_weights

array([[-4.61787768e-02, -5.48860654e-02,  2.67150551e-02, ...,
        -5.66372089e-02, -2.89367996e-02, -6.80286959e-02],
       [-1.18728654e-04, -5.02919964e-02, -3.52510368e-03, ...,
        -5.84064871e-02,  1.92214884e-02, -5.28771840e-02],
       [-3.23857330e-02,  2.07276326e-02,  6.30848706e-02, ...,
        -5.11997901e-02,  1.00646392e-02, -3.95852178e-02],
       ...,
       [ 2.19375603e-02, -1.56223550e-02,  8.53274018e-03, ...,
         2.84537673e-05,  2.59932913e-02, -2.61625051e-02],
       [-8.47661123e-02, -1.19105130e-02,  6.73796088e-02, ...,
        -5.06780967e-02,  8.05179700e-02, -5.38988039e-02],
       [-1.12659477e-01, -3.59824151e-02,  6.12834841e-02, ...,
        -7.79217556e-02,  1.37827601e-02, -6.38260916e-02]], dtype=float32)

visualizing weights of emebd layer

tensorflow has tool called projector:

And tf also has incredible guide on word embeddings

https://www.tensorflow.org/text/guide/word_embeddings

In [50]:
#crreating embedding filr ( from tf documentation)
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [51]:
# download files from colab to uplaod projector

# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

## Recurrent nueral network (RNN's)
it is useful for sequence of data

the premise rnn is to use the representation of a previous input to
aid represenatation of later input

### model 2: LSTM;
RNN componenet

LSTM= long short term memeory (one of the most popular LSTM cells)

In [58]:
 # create lstm model

from tensorflow.keras import layers
inputs=layers.Input(shape=(1,), dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
# print(x.shape)
# x=layers.LSTM( 64, return_sequences=True)(x)
# print(x.shape)
x=layers.LSTM(64)(x)
# print(x.shape)
# x=layers.Dense(1,activation="sigmoid")(x)
outputs=layers.Dense(1,activation="sigmoid")(x)

model_2=tf.keras.Model(inputs,outputs,name="model_2")

In [59]:
# get suumary
model_2.summary()

Model: "model_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,329,473 (5.07 MB)

 Trainable params: 1,329,473 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

In [61]:
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [64]:
# fit
history_2= model_2.fit(train_sentences,
                       train_label,
                       epochs=5,
                       validation_data=(val_sentences,val_labels),
                       callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                              "model_2")]
                       )

Saving TensorBoard log files to: model_logs/model_2/20250905-171746
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 11s 30ms/step - accuracy: 0.9153 - loss: 0.2852 - val_accuracy: 0.7782 - val_loss: 0.5727
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9436 - loss: 0.1576 - val_accuracy: 0.7822 - val_loss: 0.6158
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.9530 - loss: 0.1252 - val_accuracy: 0.7743 - val_loss: 0.7064
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9653 - loss: 0.1031 - val_accuracy: 0.7808 - val_loss: 0.7871
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.9687 - loss: 0.0853 - val_accuracy: 0.7822 - val_loss: 0.8376


In [65]:
# make prediction with LSTM model
model_2_pred_probs=model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step


array([[0.02818574],
       [0.8963322 ],
       [0.9995403 ],
       [0.12836216],
       [0.00100631],
       [0.9989566 ],
       [0.8557302 ],
       [0.99977535],
       [0.99960715],
       [0.4875733 ]], dtype=float32)

In [67]:
# conver model 2 prd probs to labels
model_2_preds= tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [71]:
model_2_results=calculate_results(val_labels,model_2_preds)

model_2_results

{'acuuracy': 78.21522309711287,
 'precision': 0.782423264149605,
 'recall': 0.7821522309711286,
 'f1': 0.7810542771172694}

### model 3: GRU
gru layer or gru cell (same as LSTM cell and layer)

abother RNN component

has less parameter comapared to LSTM

In [84]:
# Biuid RNN cell using GRU cell
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)

x=layers.GRU(64,return_sequences=True)(x) # to stack top of each other
# x=layers.LSTM(64,return_sequences=True)(x)
x=layers.GRU(64)(x)
# x=layers.Dense(64, activation="relu")(x)
outputs=layers.Dense(1,activation="sigmoid")(x)

model_3=tf.keras.Model(inputs,outputs,name="model_3")

In [85]:
model_3.summary()

Model: "model_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)     │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_16 (GRU)                    │ (None, 15, 64)         │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_17 (GRU)                    │ (None, 64)             │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,342,273 (5.12 MB)

 Trainable params: 1,342,273 (5.12 MB)

 Non-trainable params: 0 (0.00 B)

In [86]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [87]:
history_3= model_3.fit(train_sentences,
                       train_label,
                       epochs=5,
                       validation_data=(val_sentences,val_labels),
                       callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                              "model_3_GRU")]
                       )

Saving TensorBoard log files to: model_logs/model_3_GRU/20250905-184912
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 17s 51ms/step - accuracy: 0.8985 - loss: 0.2316 - val_accuracy: 0.7730 - val_loss: 0.6586
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 19s 44ms/step - accuracy: 0.9733 - loss: 0.0804 - val_accuracy: 0.7730 - val_loss: 0.6772
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.9777 - loss: 0.0627 - val_accuracy: 0.7730 - val_loss: 0.8191
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - accuracy: 0.9733 - loss: 0.0618 - val_accuracy: 0.7743 - val_loss: 1.2124
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - accuracy: 0.9805 - loss: 0.0503 - val_accuracy: 0.7782 - val_loss: 1.2860


In [88]:
model_3_pred_probs=model_3.predict(val_sentences)

24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step


In [90]:
model_3_pred_probs[:10]

array([[5.7269640e-02],
       [6.9764584e-01],
       [9.9969077e-01],
       [6.3811010e-03],
       [2.4077461e-04],
       [9.9959296e-01],
       [9.9756151e-01],
       [9.9976534e-01],
       [9.9970633e-01],
       [9.2797387e-01]], dtype=float32)

In [91]:
# conver model 3 pred probs to label
model_3_preds=tf.squeeze(tf.round(model_3_pred_probs))

In [93]:
model_3_results=calculate_results(val_labels,model_3_preds)
model_3_results

{'acuuracy': 77.82152230971128,
 'precision': 0.7778787744509671,
 'recall': 0.7782152230971129,
 'f1': 0.777818684489412}